# CheXNet Baseline Implementation

This notebook implements the baseline CheXNet model using a DenseNet-121 architecture pre-trained on ImageNet. This serves as the benchmark for performance.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Check device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Data Preparation

In [ ]:
# Configuration
BASE_DIR = '.'
CSV_PATH = os.path.join(BASE_DIR, 'Data_Entry_2017_v2020.csv')
IMAGE_DIRS = [
    os.path.join(BASE_DIR, 'images'),
    os.path.join(BASE_DIR, 'images 2')
]
BATCH_SIZE = 16
IMG_SIZE = 224
EPOCHS = 2
LEARNING_RATE = 1e-4

# Load Data
df = pd.read_csv(CSV_PATH)

# Map images to paths
image_paths = {}
for img_dir in IMAGE_DIRS:
    if os.path.exists(img_dir):
        for fname in os.listdir(img_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths[fname] = os.path.join(img_dir, fname)

df['path'] = df['Image Index'].map(image_paths)
df = df.dropna(subset=['path'])

# Process Labels
all_labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
              'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

for label in all_labels:
    df[label] = df['Finding Labels'].map(lambda x: 1.0 if label in x else 0.0)

print(f"Total images: {len(df)}")
print(f"Labels: {all_labels}")

In [ ]:
# Train/Val Split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")

In [ ]:
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = all_labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = row['path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        label = torch.tensor([row[l] for l in self.labels], dtype=torch.float32)
        return image, label

# Transforms (Standard CheXNet normalization)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    normalize
])

train_dataset = ChestXrayDataset(train_df, transform=train_transform)
val_dataset = ChestXrayDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

## 2. Model Definition (DenseNet-121)

In [ ]:
class CheXNet(nn.Module):
    def __init__(self, num_classes):
        super(CheXNet, self).__init__()
        self.densenet = models.densenet121(weights='DEFAULT')
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.densenet(x)

model = CheXNet(num_classes=len(all_labels)).to(device)

## 3. Training Loop

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    return running_loss / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels_np = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            
            all_preds.append(outputs.cpu().numpy())
            all_labels_np.append(labels.cpu().numpy())
            
    all_preds = np.concatenate(all_preds)
    all_labels_np = np.concatenate(all_labels_np)
    
    # Calculate AUC for each class
    aucs = []
    for i in range(len(all_labels)):
        try:
            auc = roc_auc_score(all_labels_np[:, i], all_preds[:, i])
            aucs.append(auc)
        except ValueError:
            aucs.append(0.5) # Handle cases with only one class present
            
    mean_auc = np.mean(aucs)
    return running_loss / len(loader.dataset), mean_auc

print("Starting training...")
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_auc = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Mean AUC: {val_auc:.4f}")

torch.save(model.state_dict(), 'chexnet_baseline.pth')
print("Model saved to chexnet_baseline.pth")